In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import svd
%matplotlib widget

In [ ]:
df = pd.read_csv('../csv/MSFT_M1_202402291729_202503272108.csv', sep='\t')  # Use tab as the separator
df['Datetime'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])  # Use the correct column names
df.set_index('Datetime', inplace=True)
df.drop(columns=['<DATE>', '<TIME>'], inplace=True)  # Drop the original columns
# Rename columns to match what backtrader expects (optional, if needed)
df.rename(columns={'<OPEN>': 'Open', '<HIGH>': 'High', '<LOW>': 'Low', '<CLOSE>': 'Close', '<TICKVOL>': 'TickVol', '<VOL>': 'Volume'}, inplace=True)

In [ ]:
def ssa_decompose(series, window, center_columns=False, normalize_columns=False):
    N = len(series)
    K = N - window + 1
    X = np.column_stack([series[i:i+K] for i in range(window)])  # shape K x M (but easier this way)
    X = X.T  # M x K to follow article (columns = window positions)
    # Center / normalize along columns (each row in X.T)
    if center_columns:
        X = X - X.mean(axis=0, keepdims=True)
    if normalize_columns:
        std = X.std(axis=0, ddof=0, keepdims=True)
        std[std == 0] = 1.0
        X = X / std
    # SVD
    U, s, Vt = svd(X, full_matrices=False)
    d = s**2
    return U, s, Vt, d, X

def reconstruct(U, s, Vt, indices):
    # reconstruct using selected component indices
    return (U[:, indices] * s[indices]) @ Vt[indices, :]

def diagonal_averaging(matrix_T_K):
    # matrix shape: M x K, need to average over anti-diagonals
    M, K = matrix_T_K.shape
    N = M + K - 1
    recon = np.zeros(N)
    counts = np.zeros(N)
    for i in range(M):
        for j in range(K):
            recon[i+j] += matrix_T_K[i, j]
            counts[i+j] += 1
    return recon / counts

In [ ]:

fig, (ax_price, ax_vol) = plt.subplots(
    nrows=2,
    ncols=1,
    sharex=True,
    figsize=(10, 6),
    gridspec_kw={'height_ratios': [3, 1]}
)

# верхний график цен
ax_price.plot(df['High'].to_numpy(), label='High')
ax_price.plot(df['Low'].to_numpy(),  label='Low')
ax_price.set_ylabel('Price')
ax_price.legend(loc='upper left')

# нижний график объёмов (столбцы)
ax_vol.plot(df['Volume'].to_numpy(),  label='Volume')
# ax_vol.bar(df.index, df['Volume'])
# ax_vol.set_ylabel('Volume')

# функция-обработчик: пересчитать Y-границы для объёмов
# def on_xlims_change(ax):
#     # relim пересчитывает границы исходя из видимой части данных
#     ax_vol.relim()
#     ax_vol.autoscale_view()

# # вешаем её на событие изменения X-лимитов
# ax_price.callbacks.connect('xlim_changed', on_xlims_change)
# ax_vol.callbacks.connect(  'xlim_changed', on_xlims_change)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import svd

def ssa_decompose(series, window, center=False, normalize=False):
    N = len(series)
    K = N - window + 1
    X = np.vstack([series[i:i+K] for i in range(window)])
    means = np.zeros((window,1))
    stds = np.ones((window,1))
    if center:
        means = X.mean(axis=1, keepdims=True)
        X = X - means
    if normalize:
        stds = X.std(axis=1, keepdims=True, ddof=0)
        stds[stds==0] = 1.0
        X = X / stds
    U, s, Vt = svd(X, full_matrices=False)
    return dict(U=U, s=s, Vt=Vt, means=means, stds=stds, window=window)

def reconstruct_trend(obj, idx):
    U, s, Vt = obj['U'], obj['s'], obj['Vt']
    means, stds = obj['means'], obj['stds']
    M = obj['window']
    Xn_hat = (U[:, idx] * s[idx]) @ Vt[idx, :]
    X_hat = Xn_hat * stds + means
    M, K = X_hat.shape
    N = M + K - 1
    recon = np.zeros(N)
    counts = np.zeros(N)
    for i in range(M):
        for j in range(K):
            recon[i+j] += X_hat[i,j]
            counts[i+j] +=1
    return recon / counts
# ---------- your parameters ----------
series_all = df['Close'][:3000].to_numpy()
highs = df['High'][:3000].to_numpy()
lows = df['Low'][:3000].to_numpy()

window      = 60
k_trend     = list(range(5))
initial_len = 300
n_updates   = 100

plt.figure(figsize=(9, 6))

trends      = []
last_times  = []
last_vals   = []
last_times2  = []
last_vals2   = []

# ── расчёт всех вариантов тренда ─────────────────────────────────────────
for i in range(n_updates):
    end = initial_len + i 
    if end > len(series_all):
        break
    segment = series_all[:end]

    ssa_obj = ssa_decompose(segment, window)
    trend   = reconstruct_trend(ssa_obj, k_trend)

    trends.append(trend)
    last_times.append(end - 1)   # индекс последней точки
    last_vals.append(trend[-1])  # её значение
    last_times2.append(end - 5)   # индекс последней точки
    last_vals2.append(trend[-5])  # её значение

# ── отрисовка: только «хвосты» по 10 точек ───────────────────────────────
for trend in reversed(trends):
    tail_len = min(len(trend), 10)          # вдруг ещё нет 10 точек
    t_idx    = np.arange(len(trend) - tail_len, len(trend))
    plt.plot(t_idx, trend[-tail_len:], linewidth=2)

# пунктиром – цены на момент последней итерации
final_end = initial_len + (len(trends) - 1)
plt.plot(np.arange(final_end), series_all[:final_end],
         linestyle="--", linewidth=1, label="Close price")
plt.plot(np.arange(final_end), highs[:final_end],
         linestyle="--", linewidth=1, label="High price")
plt.plot(np.arange(final_end), lows[:final_end],
         linestyle="--", linewidth=1, label="Low price")

# толстая чёрная линия – траектория «края» сглаживания
plt.plot(last_times, last_vals, color='black', linewidth=2,
         label="trend end-points trajectory")
# толстая чёрная линия – траектория «края» сглаживания
plt.plot(last_times2, last_vals2, color='red', linewidth=2,
         label="trend end-points trajectory")

plt.title("Online SSA trend: 10-точечные хвосты пересчитанных трендов")
plt.xlabel("Time index")
plt.ylabel("Price")
plt.grid(True)
plt.xlim(300, final_end)  # Zoom in to points after the first 300
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import svd

# ---------- SSA helpers --------------------------------------------------
def ssa_decompose(series, window):
    K = len(series) - window + 1
    X = np.vstack([series[i:i+K] for i in range(window)])
    U, s, Vt = svd(X, full_matrices=False)
    return U, s, Vt, window

def reconstruct_trend(U, s, Vt, window, idx):
    if isinstance(idx, int):
        idx = [idx]
    Xn = (U[:, idx] * s[idx]) @ Vt[idx, :]
    M, K = Xn.shape
    N = M + K - 1
    recon = np.zeros(N)
    counts = np.zeros(N)
    for i in range(M):
        recon[i:i+K] += Xn[i]
        counts[i:i+K] += 1
    return recon / counts

# ---------- parameters ---------------------------------------------------
series_all = df['Close'].to_numpy()

window      = 60
k_trend     = list(range(5))
initial_len = 3000
step        = 1
n_updates   = 10000

last_vals  = []
last_times = []

# ---------- build "black line" ------------------------------------------
from tqdm.notebook import tqdm
for i in tqdm(range(n_updates)):
    end = initial_len + i * step
    if end > len(series_all):
        break
    seg = series_all[:end]
    U, s, Vt, win = ssa_decompose(seg, window)
    trend = reconstruct_trend(U, s, Vt, win, k_trend)
    last_times.append(end - 1)
    last_vals.append(trend[-1])
    if i % 10 == 0:
        print(f"i = {i}, end = {end}, trend[-1] = {trend[-1]:.2f}", flush=True)

last_vals  = np.array(last_vals)
last_times = np.array(last_times)

# ---------- trading signals ---------------------------------------------
price   = series_all[last_times]
ret     = np.diff(price) / price[:-1]          # simple returns
slope   = np.diff(last_vals)

pos      = np.zeros_like(ret)
cur_pos  = 1 if slope[0] >= 0 else -1
pos[0]   = cur_pos
flips    = 0


for i in tqdm(range(1, len(ret))):
    if slope[i] * slope[i-1] < 0:              # смена знака ∂Y
        cur_pos *= -1
        flips   += 1
    pos[i] = cur_pos

str_ret   = pos * ret
equity    = np.cumprod(1 + str_ret)
bh_equity = np.cumprod(1 + ret)

# ---------- plot ---------------------------------------------------------
plt.figure(figsize=(10, 5))
plt.plot(equity,     label="SSA-reversal strategy")
plt.plot(bh_equity,  label="Buy & hold")
plt.title("Equity curve: flip on black-line reversals")
plt.xlabel("Trade index")
plt.ylabel("Cumulative growth")
plt.grid(True); plt.legend(); plt.tight_layout()
plt.show()

print(f"Total return strategy: {equity[-1]-1:.2%}")
print(f"Total return buy&hold: {bh_equity[-1]-1:.2%}")
print(f"Number of flips (trades): {flips}")


In [ ]:
price   = series_all[last_times]
ret     = np.diff(price) / price[:-1]          # simple returns
slope   = np.diff(last_vals)

# ...existing code...
pos      = np.zeros_like(ret)
cur_pos  = 1 if slope[0] >= 0 else -1
pos[0]   = cur_pos
pos[1]   = cur_pos  # initialize the second position as well
flips    = 0

for i in tqdm(range(2, len(ret))):
    # Flip position only if the sign of the slope changes (using only past info)
    if slope[i-1] * slope[i-2] < 0:
        cur_pos *= -1
        flips += 1
    pos[i] = cur_pos
# ...existing code...

str_ret   = pos * ret
equity    = np.cumprod(1 + str_ret)
bh_equity = np.cumprod(1 + ret)

# ---------- plot ---------------------------------------------------------
plt.figure(figsize=(10, 5))
plt.plot(equity,     label="SSA-reversal strategy")
plt.plot(bh_equity,  label="Buy & hold")
plt.title("Equity curve: flip on black-line reversals")
plt.xlabel("Trade index")
plt.ylabel("Cumulative growth")
plt.grid(True); plt.legend(); plt.tight_layout()
plt.show()

print(f"Total return strategy: {equity[-1]-1:.2%}")
print(f"Total return buy&hold: {bh_equity[-1]-1:.2%}")
print(f"Number of flips (trades): {flips}")


In [ ]:
window      = 60
k_trend     = list(range(5))
plt.figure(figsize=(9, 6))

def ssa_decompose(series, window, center=False, normalize=False):
    N = len(series)
    K = N - window + 1
    X = np.vstack([series[i:i+K] for i in range(window)])
    means = np.zeros((window,1))
    stds = np.ones((window,1))
    if center:
        means = X.mean(axis=1, keepdims=True)
        X = X - means
    if normalize:
        stds = X.std(axis=1, keepdims=True, ddof=0)
        stds[stds==0] = 1.0
        X = X / stds
    U, s, Vt = svd(X, full_matrices=False)
    return dict(U=U, s=s, Vt=Vt, means=means, stds=stds, window=window)

def reconstruct_trend(obj, idx):
    U, s, Vt = obj['U'], obj['s'], obj['Vt']
    means, stds = obj['means'], obj['stds']
    M = obj['window']
    Xn_hat = (U[:, idx] * s[idx]) @ Vt[idx, :]
    X_hat = Xn_hat * stds + means
    M, K = X_hat.shape
    N = M + K - 1
    recon = np.zeros(N)
    counts = np.zeros(N)
    for i in range(M):
        for j in range(K):
            recon[i+j] += X_hat[i,j]
            counts[i+j] +=1
    return recon / counts

segment_len = 3000
segment = df['Close'][-segment_len:].to_numpy()  # последние 300 точек
ssa_obj = ssa_decompose(segment, window)
trend   = reconstruct_trend(ssa_obj, k_trend)
plt.plot(np.arange(len(df['Close'])-segment_len, len(df['Close'])), trend, color="red", linewidth=2)

segment_len = 150
segment = df['Close'][-segment_len:].to_numpy()  # последние 150 точек
ssa_obj = ssa_decompose(segment, window)
trend   = reconstruct_trend(ssa_obj, k_trend)
plt.plot(np.arange(len(df['Close'])-segment_len, len(df['Close'])), trend, color="green", linewidth=2)

segment_len = 3000
highs = df['High'][-segment_len:].to_numpy()
lows = df['Low'][-segment_len:].to_numpy()
plt.plot(np.arange(len(df['High'])-segment_len, len(df['High'])), highs, color='gray', linestyle="--", linewidth=1, label="High price")
plt.plot(np.arange(len(df['Low'])-segment_len, len(df['Low'])), lows, color='gray', linestyle="--", linewidth=1, label="Low price")

plt.tight_layout()
plt.show()
